<a href="https://colab.research.google.com/github/johncoogan53/DL_HW3/blob/main/labllm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune GPT-2 on wiki-text

In this Lab, we are using a series of library from Hugging Face (i.e. tranformers, datasets, peft). You may need to go through the document of these library to learn the usage. (Hint: you may use the imported contents in the code cell below, other contents is not necessary for this lab)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# for google colab
!pip install transformers
!pip install datasets
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.2 MB/s eta 0:00:00


In [3]:
import torch
print(torch.__version__)

2.5.0+cu121


In [4]:
import os
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from datasets import load_dataset

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling

from torch.utils.data import DataLoader
import torch.nn as nn

cuda


In [5]:
torch.cuda.empty_cache()


## Lab 2(a) Generate text with GPT2

Using the API provided by hugging face, we can easily load the pre-trained GPT2 model and generate text. (GPT2 is a early generative model, the quality of the generated text is not as good as the later model like GPT3.)

In [6]:
# Print the value of HF_DATASETS_CACHE
print(f"HF_DATASETS_CACHE: {os.getenv('HF_DATASETS_CACHE')}")
# os.environ['HF_DATASETS_CACHE'] = "D:\\MIDS\\Deep_Learning\\HW3\\resources"
# print(f"HF_DATASETS_CACHE: {os.getenv('HF_DATASETS_CACHE')}")

HF_DATASETS_CACHE: None


In [7]:
# your code here: load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

In [27]:

def generate_text(model, tokenizer, prompt, max_length):

    # your code here: tokenize the prompt
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # your code here: generate token using the model
    gen_tokens = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, return_dict_in_generate=True)
    generated_ids = gen_tokens['sequences']
    # your code here: decode the generated tokens
    gen_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(gen_text)

generate_text(model, tokenizer, "GPT-2 is a langugae model based on transformer developed by OpenAI", 100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 is a langugae model based on transformer developed by OpenAI. It is a simple, fast, and scalable model of the human brain. It is based on the concept of the "brain as a machine".

The model is based on the concept of the "brain as a machine". The model is based on the concept of the "brain as a machine". The model is based on the concept of the "brain as a machine". The model is based on the


## Lab 2(b) Prepare dataset for training

Please fill the code cell below to download the dataset and prepare the dataset for finetuning.


In [8]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# your code here: load the dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Print the size of the dataset in bytes
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [9]:

# get 10% of dataset
dataset_train = dataset["train"].select(range(len(dataset["train"]) // 10))
dataset_valid = dataset["validation"].select(range(len(dataset["validation"]) // 10))
print(dataset_valid.shape)
print(dataset_train.shape)
del dataset #drop the full dataset from memory since we have memory issues later


(376, 1)
(3671, 1)


In [10]:

# your code here: implement function that tokenize the dataset and set labels to be the same as input_ids
def tokenize_function(examples,tokenizer=tokenizer):
    import torch #workers may not have torch in scope
    tokenized = tokenizer(examples["text"],padding='max_length', truncation=True)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


# your code here: tokenize the dataset (you may need to remove columns that are not needed)
tokenized_datasets_train = dataset_train.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_datasets_valid = dataset_valid.map(tokenize_function, batched=True, remove_columns=["text"])


tokenized_datasets_train.set_format("torch")
tokenized_datasets_valid.set_format("torch")

# your code here: create datacollator for training and validation dataset
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)#base arguments, could be wrong

train_dataloader = DataLoader(tokenized_datasets_train, shuffle=True, batch_size=4, collate_fn=data_collator)
valid_dataloader = DataLoader(tokenized_datasets_valid, batch_size=4, collate_fn=data_collator)

# Test the DataLoader
for batch in train_dataloader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['labels'].shape)
    break

print("DataLoader is working correctly!")

Map:   0%|          | 0/3671 [00:00<?, ? examples/s]

Map:   0%|          | 0/376 [00:00<?, ? examples/s]

torch.Size([4, 1024])
torch.Size([4, 1024])
torch.Size([4, 1024])
DataLoader is working correctly!


## Lab 2(c) Evaluate perplexity on wiki-text

Before finetuning, we evaluate the pre-trained GPT2 model on the wiki-text dataset. The perplexity is a common metric to evaluate the performance of language model. The lower the perplexity, the better the model. To compute the perplexity in practice, we use the formula as follows, which is a transformation of the formula in class:
$PP(W) = \exp\left(-\frac{1}{N} \sum_{i=1}^{N} \log P(w_i|\text{context})\right)$

In [28]:
def evaluate_perplexity(model, dataloader):
    model.eval()
    total_loss = 0
    total_length = 0
    loss_fn = nn.CrossEntropyLoss(reduction='sum')

    with torch.no_grad():
        for batch in dataloader:
            # your code here: get the input_ids, attention_mask, and labels from the batch
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # your code here: forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # your code here: calculate the loss
            loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            total_loss += loss.item()
            total_length += attention_mask.sum().item()

    # Calculate perplexity
    perplexity = torch.exp(torch.tensor(total_loss / total_length))

    return perplexity.item()


perplexity = evaluate_perplexity(model, valid_dataloader)
print(f"Initial perplexity: {perplexity}")

Initial perplexity: 42.9958610534668


## Lab 2(d) Fine-tune GPT2 on wiki-text



In [ ]:
#clear cache
#torch.cuda.empty_cache()

In [14]:

# Set up training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DLHW3/gpt2-wikitext-2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    prediction_loss_only=True,
    # your code here: report validation and training loss every epoch
    logging_strategy="epoch",
    evaluation_strategy="epoch",
)

# your code here: create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,
)

trainer.train()
trainer.save_model()



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,3.618800,3.356315
2,2.983200,3.388046
3,2.691600,3.432857


# Test fine-tuned model

In [15]:
# your code here: load the fine-tuned model
model_finetuned = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/DLHW3/gpt2-wikitext-2")
model_finetuned.to(device)
perplexity = evaluate_perplexity(model_finetuned, valid_dataloader)
print(f"fine-tuned perplexity: {perplexity}")

fine-tuned perplexity: 28.597455978393555


# Generate some text using the fine-tuned model

In [16]:
# load the fine-tuned model

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# generate text
generate_text(
    model_finetuned,
    tokenizer,
    "GPT-2 is a langugae model based on transformers developed by OpenAI",
    100,
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 is a langugae model based on transformers developed by OpenAI, and is used in the development of the CATASAR ( CATASAR Advanced CATASAR ) CATASAR system. The CATASAR system is a multi @-@ stage system that uses a CATASAR array to measure the strength of the magnetic field. The CATASAR system is based on the CATASAR Advanced CATASAR ( CATASAR Advanced CATASAR ) CATAS


## Lab 2(e) Parameter efficient fine-tuning (LoRA)

finetune the base gpt model through LoRA

In [12]:
print(torch.cuda.memory_allocated())  # Allocated memory in bytes
print(torch.cuda.max_memory_allocated())
torch.cuda.empty_cache()
#del model_finetuned
#del model
print(torch.cuda.memory_allocated())  # Allocated memory in bytes
print(torch.cuda.max_memory_allocated())

0
0
0
0


In [15]:
from peft import LoraConfig, LoraModel

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# your code here: load GPT2 model and add the lora adapter
model_l = AutoModelForCausalLM.from_pretrained("openai-community/gpt2").to(device)


model_lora = LoraModel(model_l, peft_config,"default").to(device)


training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DLHW3/gpt2-lora-wikitext-2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=400,
    warmup_steps=500,
    save_strategy="no",
    prediction_loss_only=True,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    remove_unused_columns=False,
)

# your code here: set trainer and train the model
trainer = Trainer(
    model=model_lora,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,

)

trainer.train()




/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,4.225100,No log
2,3.882100,No log
3,3.794400,No log


TrainOutput(global_step=1377, training_loss=3.9672104247514977, metrics={'train_runtime': 1409.1419, 'train_samples_per_second': 7.815, 'train_steps_per_second': 0.977, 'total_flos': 5914857399386112.0, 'train_loss': 3.9672104247514977, 'epoch': 3.0})

In [20]:
def untie_weights(model):
    # Detach and clone the weights, then reassign as parameters
    model.model.lm_head.weight = torch.nn.Parameter(model.model.lm_head.weight.detach().clone())
    model.model.transformer.wte.weight = torch.nn.Parameter(model.model.transformer.wte.weight.detach().clone())


In [21]:
untie_weights(model_lora)#to prevent tensor share
trainer.save_model()

ppl = evaluate_perplexity(model_lora, valid_dataloader)
print(f"Perplexity after lora finetuning: {ppl}")

Perplexity after lora finetuning: 30.276874542236328


# Evaluate lora fine-tuned model on wiki-text

compare the text generated by the fully fine-tuned model and LoRA fine-tuned model and the pre-trained model. Do you see any difference in the quality of the generated text? Try to explain why. (Hint: trust your result and report as it is.)

In [22]:
generate_text(
    model_finetuned_lora,
    tokenizer,
    "GPT-2 is a langugae model based on transformers developed by OpenAI",
    100,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2 is a langugae model based on transformers developed by OpenAI, which is based on the concept of a "transformation" of the human brain into a "transformation" of the animal brain. The model is based on the concept of a "transformation" of the human brain into a "transformation" of the animal brain, and is based on the concept of a "transformation" of the human brain into a "transformation" of the animal brain


Compare the perplexity of the fully fine-tuned model and LoRA fine-tuned model. Do you see any difference in the perplexity? Try to explain why.

In [ ]:
ppl = evaluate_perplexity(model_lora, valid_dataloader)

print(f"Perplexity after lora finetuning: {ppl}")